# Aim

In step four, I'd like to address expanding the phylogeny with more R2R3 MYB subfamiles to address some open questions: 
* Can I add a subfamily label to the other Azfi sequences I found before with a custom hmm.
* Can I infer the phylogeny to more confidently/explicitly draw the Salvinales VII branch as a part of the remaining VII branch by including other subfamiles and an outgroup

In a later notebook, I'll address adding extra data like: 
Next steps: 

* Can I add intron/exon structures as a second line of evidence supporting the different subfamilies
* Finally, I'd like to log2 expressen/fold change to the Azfi sequences; to illustrate how these subfamilies change expression upon transistion to sexual reproduction at least in _Azolla_

Since retrieving sequences from the different databases is quite a pain, 
I'll first retrieve only NCBI listed sequences (which I can retrieve in batch).
Subsequently, I may reitterate in this notebook, expanding on that analysis with sequences from other genome databases.